In [1]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/AppleGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='AppleGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'AppleGothic'}) # 폰트 설정
plt.rc('font', family='AppleGothic')

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns

In [2]:
# 사용자 라이브러리 추가

from utils import (
    xbar_s_chart,
    xbar_r_chart,
    capability_plot_boxcox,
    capability_analysis,
    within_variation,
    capability_plot,
    capability_analysis_boxcox,
    gage_rr_final,
    nelson_rules,
    i_mr_chart,
    spc_constants,
)

In [3]:
food = pd.read_csv('data/food_clean+remove_outlier.csv')
food

,Lot_ID,Line,Shift,Operator,Temp,Humidity,Weight,Defect,Defect_Type
0,L003,A,주간,O8,26.4,42.9,99.92,0.0,양품
1,L029,B,주간,O2,24.3,57.7,101.77,0.0,양품
2,L035,A,주간,O4,23.5,44.3,99.82,0.0,양품
3,L039,C,주간,O3,24.4,38.2,100.60,0.0,양품
4,L018,A,야간,O9,26.5,44.4,99.76,0.0,양품
...,...,...,...,...,...,...,...,...,...
367,L046,A,야간,O2,23.1,58.1,100.69,0.0,양품
368,L013,A,주간,O9,25.5,53.0,99.55,0.0,양품
369,L027,A,주간,O5,24.2,46.3,100.86,0.0,양품
370,L047,C,야간,O5,24.0,50.7,98.69,0.0,양품


In [19]:
df_grr = pd.read_csv('data/gage_rr_sample.csv')
df_grr.head()

,Part,Operator,Trial,Weight
0,P1,O1,1,100.26
1,P1,O1,2,100.26
2,P1,O2,1,100.30
3,P1,O2,2,99.87
4,P1,O3,1,99.80


In [18]:
# Gage R&R 분석
from utils.gage_rr_final import grr

grr(df_grr, op='Operator', part='Part', measure='Weight', tol=2.0)


[1] 분산분석표 (Random Effects Model)
                           SS  df      MS        F  p-value
Operator               0.9456   2  0.4728  15.4418   0.0001
Part                   8.1044   9  0.9005  29.4102   0.0000
Operator*Part          0.5511  18  0.0306   0.9759   0.5087
Repeatability (Error)  0.9412  30  0.0314      NaN      NaN

[2] 분산 성분
                               Variance Component  Percent Contribution
Total Gage R&R                           0.053484             26.949155
Repeatability                            0.031375             15.808973
Reproducibility (Operator)               0.022109             11.140182
Reproducibility (Interaction)            0.000000              0.000000
Part-to-Part                             0.144979             73.050845
Total                                    0.198463            100.000000

[3] 연구 변동 및 공정 평가
                                   SD  Study Variation (6σ)  %Study Variation  %Tolerance (Tol = 2.0)
Total Gage R&R                

## 결과 해석

[1] 분산분석표
- Operator * Part : p-value가 0.05보다 크므로 귀무가설(교호작용이 존재한다.) 채택

[2] 분산 성분
- Total GageR&R : %기여(%Percent Contribution)이 26.949155% 이므로 조건부 합격 기준인 9%보다 크다 (= 측정 시스템 개선이 필요하다, 공정관리/개선 부적합)

[3] 연구 변동 및 공정 평가
- Total GageR&R : %연구변동(%Study Varication)이 51.9126% 이므로 조건부 합격 기준인 30%보다 크다 (= 측정 시스템 개선이 필요하다, 공정관리/개선 부적합)
- Total GageR&R : %공차(%Tolerance)이 69.3799% 이므로 조건부 합격 기준인 30%보다 크다 (= 측정 시스템 개선이 필요하다, 합/부 판정에 부적합)

반복성(Repeatability) > 재현성(Reproducibility)이므로 계측기 자체의 문제로 보여진다. 따라서 계측기 보전 및 계측기 구조 재검토의 필요성이 보인다.